<a href="https://colab.research.google.com/github/KORALLLL/MTUCI_EMNIST/blob/Sasha/ColabFiles/NewExperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import random
import numpy as np
import torchvision.datasets
import torchvision.transforms as transforms
from tqdm import tqdm_notebook as bar
import pickle
import torchvision.datasets

random.seed(0)
np.random.seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
!git clone https://github.com/KORALLLL/MTUCI_EMNIST.git

Cloning into 'MTUCI_EMNIST'...
remote: Enumerating objects: 36499, done.
remote: Counting objects: 100% (12859/12859), done.
remote: Compressing objects: 100% (12692/12692), done.
remote: Total 36499 (delta 186), reused 12804 (delta 159), pack-reused 23640
Receiving objects: 100% (36499/36499), 128.89 MiB | 11.92 MiB/s, done.
Resolving deltas: 100% (471/471), done.


In [3]:
emnist_train = torchvision.datasets.EMNIST('./', split='letters', download = True, train = True)
mnist_train = torchvision.datasets.EMNIST('./', split='mnist', download = True, train = True)
emnist_val = torchvision.datasets.EMNIST('./', split='letters', download = True, train = False)
mnist_val = torchvision.datasets.EMNIST('./', split='mnist', download = True, train = False)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

emnist_train_data = emnist_train.data
mnist_train_data = mnist_train.data
train_data = torch.cat([emnist_train_data[:24960], mnist_train_data[:12000]], dim=0).float().unsqueeze(1).to(device) / 255

emnist_val_data = emnist_val.data
mnist_val_data = mnist_val.data
val_data = torch.cat([emnist_val_data, mnist_val_data], dim=0).float().unsqueeze(1).to(device) / 255

temp_emnist_train_labels = emnist_train.targets[:24960]
emnist_train_labels = []

temp_emnist_val_labels = emnist_val.targets
emnist_val_labels = []

for i in range(len(temp_emnist_train_labels)):
  if temp_emnist_train_labels[i]==15:
    emnist_train_labels.append(torch.tensor(0))
  elif temp_emnist_train_labels[i]>15:
    emnist_train_labels.append(temp_emnist_train_labels[i]+8)
  else:
    emnist_train_labels.append(temp_emnist_train_labels[i]+9)

for i in range(len(temp_emnist_val_labels)):
  if temp_emnist_val_labels[i]==15:
    emnist_val_labels.append(torch.tensor(0))
  elif temp_emnist_val_labels[i]>15:
    emnist_val_labels.append(temp_emnist_val_labels[i] + 8)
  else:
    emnist_val_labels.append(temp_emnist_val_labels[i]+9)

mnist_train_labels = mnist_train.targets
train_labels = torch.cat([torch.stack(emnist_train_labels[:24960]), mnist_train_labels[:12000]], dim=0).to(device)

mnist_val_labels = mnist_val.targets
val_labels = torch.cat([torch.stack(emnist_val_labels), mnist_val_labels], dim=0).to(device)


file = open('MTUCI_EMNIST/dataset.pkl', 'rb')
test_dataset = pickle.load(file)
file.close()

test_data = test_dataset['data'].numpy()
test_data = np.flip(test_data, axis = 3)
test_data = np.rot90(test_data, k=1, axes=(2,3))
test_data = 1 - test_data
test_data = torch.from_numpy(test_data).float().to(device)
test_labels = test_dataset['targets'].to(device)


100%|██████████| 561753746/561753746 [00:07<00:00, 73689237.41it/s] 


Extracting ./EMNIST/raw/gzip.zip to ./EMNIST/raw


##Лучшая сеть по итогам экспериментов


In [ ]:
class LeNetMaxAvg(torch.nn.Module):
  def __init__(self):
    super(LeNetMaxAvg, self).__init__()

    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=2,
                                 kernel_size=3, padding=1)
    self.act1 = torch.nn.Tanh()
    self.conv2 = torch.nn.Conv2d(in_channels=2, out_channels=4,
                                 kernel_size=3, padding=1)

    self.act2 = torch.nn.Tanh()
    self.conv3 = torch.nn.Conv2d(in_channels=4, out_channels=8,
                                 kernel_size=3, padding=1)
    self.act3 = torch.nn.Tanh()
    self.conv4 = torch.nn.Conv2d(in_channels=8, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act4 = torch.nn.Tanh()
    self.conv5 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act5 = torch.nn.Tanh()
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act6 = torch.nn.Tanh()


    self.conv6 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act7 = torch.nn.Tanh()
    self.conv7 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act8 = torch.nn.Tanh()
    self.conv8 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act9 = torch.nn.Tanh()
    self.conv9 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=0)
    self.act10 = torch.nn.Tanh()
    self.conv10 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=0)
    self.act11 = torch.nn.Tanh()
    self.pool2 = torch.nn.AvgPool2d(kernel_size=2, stride=2)
    self.act12 = torch.nn.Tanh()

    self.fc1 = torch.nn.Linear(5 * 5 * 16, 120)
    self.act13 = torch.nn.ReLU()
    self.fc2 = torch.nn.Linear(120, 84)
    self.act14 = torch.nn.ReLU()
    self.fc3 = torch.nn.Linear(84, 35)

  def forward(self, x):
    x = self.conv1(x)
    x = self.act1(x)
    x = self.conv2(x)
    x = self.act2(x)
    x = self.conv3(x)
    x = self.act3(x)
    x = self.conv4(x)
    x = self.act4(x)
    x = self.conv5(x)
    x = self.act5(x)
    x = self.pool1(x)
    x = self.act6(x)

    x = self.conv6(x)
    x = self.act7(x)
    x = self.conv7(x)
    x = self.act8(x)
    x = self.conv8(x)
    x = self.act9(x)
    x = self.conv9(x)
    x = self.act10(x)
    x = self.conv10(x)
    x = self.act11(x)
    x = self.pool2(x)
    x = self.act12(x)

    x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

    x = self.fc1(x)
    x = self.act13(x)
    x = self.fc2(x)
    x = self.act14(x)
    x = self.fc3(x)

    return x


##Добавление слоев батч-нормализации

In [4]:
class LeNetBatch(torch.nn.Module):
  def __init__(self):
    super(LeNetBatch, self).__init__()

    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=2,
                                 kernel_size=3, padding=1)
    self.batch_norm1 = torch.nn.BatchNorm2d(2)
    self.act1 = torch.nn.Tanh()
    self.conv2 = torch.nn.Conv2d(in_channels=2, out_channels=4,
                                 kernel_size=3, padding=1)
    self.batch_norm2 = torch.nn.BatchNorm2d(4)

    self.act2 = torch.nn.Tanh()
    self.conv3 = torch.nn.Conv2d(in_channels=4, out_channels=8,
                                 kernel_size=3, padding=1)
    self.batch_norm3 = torch.nn.BatchNorm2d(8)
    self.act3 = torch.nn.Tanh()
    self.conv4 = torch.nn.Conv2d(in_channels=8, out_channels=16,
                                 kernel_size=3, padding=1)
    self.batch_norm4 = torch.nn.BatchNorm2d(16)
    self.act4 = torch.nn.Tanh()
    self.conv5 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.batch_norm5 = torch.nn.BatchNorm2d(16)
    self.act5 = torch.nn.Tanh()
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act6 = torch.nn.Tanh()


    self.conv6 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.batch_norm6 = torch.nn.BatchNorm2d(16)
    self.act7 = torch.nn.Tanh()
    self.conv7 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.batch_norm7 = torch.nn.BatchNorm2d(16)
    self.act8 = torch.nn.Tanh()
    self.conv8 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.batch_norm8 = torch.nn.BatchNorm2d(16)
    self.act9 = torch.nn.Tanh()
    self.conv9 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=0)
    self.batch_norm9 = torch.nn.BatchNorm2d(16)
    self.act10 = torch.nn.Tanh()
    self.conv10 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=0)
    self.batch_norm10 = torch.nn.BatchNorm2d(16)
    self.act11 = torch.nn.Tanh()
    self.pool2 = torch.nn.AvgPool2d(kernel_size=2, stride=2)
    self.act12 = torch.nn.Tanh()

    self.fc1 = torch.nn.Linear(5 * 5 * 16, 120)
    self.act13 = torch.nn.ReLU()
    self.fc2 = torch.nn.Linear(120, 84)
    self.act14 = torch.nn.ReLU()
    self.fc3 = torch.nn.Linear(84, 35)

  def forward(self, x):
    x = self.conv1(x)
    x = self.batch_norm1(x)
    x = self.act1(x)
    x = self.conv2(x)
    x = self.batch_norm2(x)
    x = self.act2(x)
    x = self.conv3(x)
    x = self.batch_norm3(x)
    x = self.act3(x)
    x = self.conv4(x)
    x = self.batch_norm4(x)
    x = self.act4(x)
    x = self.conv5(x)
    x = self.batch_norm5(x)
    x = self.act5(x)
    x = self.pool1(x)
    x = self.act6(x)

    x = self.conv6(x)
    x = self.batch_norm6(x)
    x = self.act7(x)
    x = self.conv7(x)
    x = self.batch_norm7(x)
    x = self.act8(x)
    x = self.conv8(x)
    x = self.batch_norm8(x)
    x = self.act9(x)
    x = self.conv9(x)
    x = self.batch_norm9(x)
    x = self.act10(x)
    x = self.conv10(x)
    x = self.batch_norm10(x)
    x = self.act11(x)
    x = self.pool2(x)
    x = self.act12(x)

    x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

    x = self.fc1(x)
    x = self.act13(x)
    x = self.fc2(x)
    x = self.act14(x)
    x = self.fc3(x)

    return x


##Добавление дропаута

In [ ]:
class LeNetMaxAvg(torch.nn.Module):
  def __init__(self, dropout_rate):
    super(LeNetMaxAvg, self).__init__()

    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=2,
                                 kernel_size=3, padding=1)
    self.act1 = torch.nn.Tanh()
    self.dropout1 = torch.nn.Dropout(p=dropout_rate)
    self.conv2 = torch.nn.Conv2d(in_channels=2, out_channels=4,
                                 kernel_size=3, padding=1)

    self.act2 = torch.nn.Tanh()
    self.dropout2 = torch.nn.Dropout(p=dropout_rate)
    self.conv3 = torch.nn.Conv2d(in_channels=4, out_channels=8,
                                 kernel_size=3, padding=1)
    self.act3 = torch.nn.Tanh()
    self.dropout3 = torch.nn.Dropout(p=dropout_rate)
    self.conv4 = torch.nn.Conv2d(in_channels=8, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act4 = torch.nn.Tanh()
    self.dropout4 = torch.nn.Dropout(p=dropout_rate)
    self.conv5 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act5 = torch.nn.Tanh()
    self.dropout5 = torch.nn.Dropout(p=dropout_rate)
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act6 = torch.nn.Tanh()


    self.conv6 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act7 = torch.nn.Tanh()
    self.dropout6 = torch.nn.Dropout(p=dropout_rate)
    self.conv7 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act8 = torch.nn.Tanh()
    self.dropout7 = torch.nn.Dropout(p=dropout_rate)
    self.conv8 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=1)
    self.act9 = torch.nn.Tanh()
    self.dropout8 = torch.nn.Dropout(p=dropout_rate)
    self.conv9 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=0)
    self.act10 = torch.nn.Tanh()
    self.dropout9 = torch.nn.Dropout(p=dropout_rate)
    self.conv10 = torch.nn.Conv2d(in_channels=16, out_channels=16,
                                 kernel_size=3, padding=0)
    self.act11 = torch.nn.Tanh()
    self.dropout10 = torch.nn.Dropout(p=dropout_rate)
    self.pool2 = torch.nn.AvgPool2d(kernel_size=2, stride=2)
    self.act12 = torch.nn.Tanh()

    self.fc1 = torch.nn.Linear(5 * 5 * 16, 120)
    self.act13 = torch.nn.ReLU()
    self.fc2 = torch.nn.Linear(120, 84)
    self.act14 = torch.nn.ReLU()
    self.fc3 = torch.nn.Linear(84, 35)

  def forward(self, x):
    x = self.conv1(x)
    x = self.act1(x)
    x = self.conv2(x)
    x = self.act2(x)
    x = self.conv3(x)
    x = self.act3(x)
    x = self.conv4(x)
    x = self.act4(x)
    x = self.conv5(x)
    x = self.act5(x)
    x = self.pool1(x)
    x = self.act6(x)

    x = self.conv6(x)
    x = self.act7(x)
    x = self.conv7(x)
    x = self.act8(x)
    x = self.conv8(x)
    x = self.act9(x)
    x = self.conv9(x)
    x = self.act10(x)
    x = self.conv10(x)
    x = self.act11(x)
    x = self.pool2(x)
    x = self.act12(x)

    x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

    x = self.fc1(x)
    x = self.act13(x)
    x = self.fc2(x)
    x = self.act14(x)
    x = self.fc3(x)

    return x


Цикл обучения

In [5]:

for j in range(1):
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  lenet5 = LeNetBatch().to(device)
  loss = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(lenet5.parameters(), lr=1.0e-3)
  batch_size = 1120
  test_accuracy_history = []
  test_loss_history = []

  for epoch in range(100):
    order = np.random.permutation(len(train_data))
    for start_index in range(0, len(train_data), batch_size):
      optimizer.zero_grad()

      batch_indexes = order[start_index:start_index + batch_size]
      train_batch = train_data[batch_indexes].to(device)
      label_batch = train_labels[batch_indexes].to(device)

      preds = lenet5.forward(train_batch)

      loss_value = loss(preds, label_batch)
      loss_value.backward()

      optimizer.step()
  test_preds = lenet5.forward(test_data)
  test_loss_history.append(loss(test_preds, test_labels).data.cpu())

  accuracy = (test_preds.argmax(dim=1) == test_labels).float().mean().data.cpu()
  test_accuracy_history.append(accuracy)
  print(accuracy)
  print("-----")

tensor(0.7280)
-----
